In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import pycuda.autoinit



cuda_bzmodule =  SourceModule("""
   #include <stdio.h>

   __device__ inline int pbc(int x, int L);
   __device__ inline double moore_mean_conc (double *arr, 
                                             int i, int j, 
                                             int n, int s, int w, int e,
                                             int nc);                                  
  

     /**************************************************************************
     *
     * bz_reaction: 
     *--------------------------------------------------------------------------
     *  Input: x = value to transform
     *         L = lattice size
     *
     *  Ouput: void
     *
     **************************************************************************/
   __global__ void bz_reaction(double *aarr_in, double *aarr_out,
                               double *barr_in, double *barr_out,
                               double *carr_in, double *carr_out,
                               double alpha, double beta, double gamma,
                               int nr, int nc)
    {
      
      int i = threadIdx.x+ blockIdx.x* blockDim.x;
      int j = threadIdx.y+ blockIdx.y* blockDim.y;

      // Coordinates in pbc
      int n = pbc(i-1,nr);
      int s = pbc(i+1,nr);
      int w = pbc(j-1,nc);
      int e = pbc(j+1,nc);
     
      // Mean value
      double ma = moore_mean_conc(aarr_in, i, j, n, s, w, e, nc);
      double mb = moore_mean_conc(barr_in, i, j, n, s, w, e, nc);
      double mc = moore_mean_conc(carr_in, i, j, n, s, w, e, nc);

      // Reaction
      aarr_out[i*nc+j] =  ma + ma*(alpha*mb - gamma*mc);
      barr_out[i*nc+j] =  mb + mb*(beta*mc -  alpha*ma);
      carr_out[i*nc+j] =  mc + mc*(gamma*ma - beta*mb);

      // Constrain the values to the interval [0,1]
      if(aarr_out[i*nc+j]>1)  aarr_out[i*nc+j] = 1;
      if(aarr_out[i*nc+j]<0)  aarr_out[i*nc+j] = 0;

      if(barr_out[i*nc+j]>1) barr_out[i*nc+j] = 1;
      if(barr_out[i*nc+j]<0) barr_out[i*nc+j] = 0;

      if(carr_out[i*nc+j]>1) carr_out[i*nc+j] = 1;
      if(carr_out[i*nc+j]<0)  carr_out[i*nc+j] = 0;

    }



     /**************************************************************************
     *
     * moore_mean_conc: Mean value around position from a 9-points Moore stencil
     *--------------------------------------------------------------------------
     *  Input: arr = input array 
     *         i = central coordiante row
     *         i = central coordiante column
     *         n = north coordinate
     *         s = south coordinate
     *         e = east coordinate
     *         w = west coordinate
     *         nc = total number of colums
     *      
     *  Ouput: m = average value around central point i,j
     *
     **************************************************************************/
    __device__ inline double moore_mean_conc (double *arr, 
                                             int i, int j, 
                                             int n, int s, int w, int e,
                                             int nc)
    {
        double m = arr[n*nc+w]  + // nw
                   arr[n*nc+j]  + // n
                   arr[n*nc+e]  + // ne
                   arr[i*nc+w]  + // w
                   arr[i*nc+j]  + // x
                   arr[i*nc+e]  + // e
                   arr[s*nc+w]  + // sw
                   arr[s*nc+j]  + // s
                   arr[s*nc+e];   // se
        return m/9.0;
    }



     /**************************************************************************
     *
     * pbc: Returns coordinates under peridoic boundary conditions
     *--------------------------------------------------------------------------
     *  Input: x = value to transform
     *         L = lattice size
     *
     *  Ouput: x in pbc 
     *
     **************************************************************************/
    __device__ inline int pbc(int x, int L)
    {
        return  x - (int)floor((double)x/L)*L;
    }





""")



def initilise_array(h, w):
  """
      initialise_array
      Generates a drop in the middle of the array

      Input:    arr = w x h array to initialise
                h = height, number of rows
                w = width, number of columns

      Ouput     arr = Output array with the drop
  """
  a = np.float64(np.random.rand(h,w))
  b = np.float64(np.random.rand(h,w))
  c = np.float64(np.random.rand(h,w))
  return a,b,c

def bzreact_dynamics(w, h, alpha, beta, gamma, maxtime):
  # We need to explicitely transform to the specific data types because   
  #  CUDA variablesare type-specific
  w=np.int32(w)
  h=np.int32(h)
  alpha = np.float64(alpha)
  beta  = np.float64(beta)
  gamma = np.float64(gamma)

  # Get the function from the CUDA module
  bzreact = cuda_bzmodule.get_function("bz_reaction") 

  # Initialisation
  aarr, barr, carr = initilise_array(h, w)


 # Allocate memory for the arrays in the GPU and copy the current initilised value
  aarr_gpu     = cuda.mem_alloc(aarr.nbytes) 
  aarr_upd_gpu = cuda.mem_alloc(aarr.nbytes)
  barr_gpu     = cuda.mem_alloc(barr.nbytes) 
  barr_upd_gpu = cuda.mem_alloc(barr.nbytes)  
  carr_gpu     = cuda.mem_alloc(carr.nbytes) 
  carr_upd_gpu = cuda.mem_alloc(carr.nbytes) 
  #cuda.memcpy_htod(aarr_gpu, aarr)
  #cuda.memcpy_htod(barr_gpu, barr)
  #cuda.memcpy_htod(carr_gpu, carr)


  # Iterate the function on the GPU 
  # We first define the minimal unit ie, the block size. A grid is then a 
  # a collection of blocks, so we attribute a number of blocks that divisible with 
  # with the block size considering the size of the array. 
  bw=32   
  bh=32
  gw=int(np.floor(w/bw))
  gh=int(np.floor(h/bh))

  for t in range(0, maxtime):
    # Send data from computer to GPU 
    cuda.memcpy_htod(aarr_gpu, aarr)
    cuda.memcpy_htod(barr_gpu, barr)
    cuda.memcpy_htod(carr_gpu, carr)

    # Reaction
    bzreact(aarr_gpu, aarr_upd_gpu,
            aarr_gpu, aarr_upd_gpu,
            aarr_gpu, aarr_upd_gpu,
            alpha, beta, gamma,
            block=(bw,bh,1), grid=(gw,gh))

    # Retrieve the data from the GPU to the computer
    cuda.memcpy_dtoh(aarr, aarr_upd_gpu)
    cuda.memcpy_dtoh(barr, barr_upd_gpu)
    cuda.memcpy_dtoh(barr, barr_upd_gpu)




# if __name__ == "__main__":
w = 512
h = 512
alpha = 1.2
beta = 1.0
gamma = 1.0
maxtime = 50
bzreact_dynamics(w, h, alpha, beta, gamma, maxtime)

ModuleNotFoundError: No module named 'pycuda'